In [4]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 


In [5]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics

In [6]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
from Model import net
from Dataloader2 import EEGDataset

importing Jupyter notebook from Dataloader2.ipynb


In [7]:
def Accuracy(y_pred, y, train_count):
    max_values, _ = torch.max(y_pred, dim=1, keepdim=True)
    mask = y_pred == max_values
    y_pred = mask.int()
    correct_num = torch.sum(torch.all(torch.eq(y, y_pred), dim=1)).item()
    accuracy = correct_num / train_count
    return accuracy

In [8]:
MNE_Data = EEGDataset(root_dir="D:\MNE Data")

In [9]:
batch_size = 1
train_dataloader = DataLoader(MNE_Data, batch_size=batch_size, shuffle=True)

In [10]:
model = net(T = 3000, C = 19, input_size = 60, hidden_size = 30, num_layers=4, spatial_num= 10, dropout=0.2, pool=1).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
epochs = 60

In [12]:
for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        log = []
        for index, data in enumerate(train_dataloader):
            
            x, y = data
            y = y.to(torch.float64)
            x = x.reshape(-1, 1, 19, 3000).float()
            y = F.one_hot(torch.tensor(torch.tensor([y.item()]).to(torch.int64)), num_classes=4).expand(x.shape[0], -1).float()
            train_count = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print(Accuracy(y_pred, y, train_count))
            log.append(Accuracy(y_pred, y, train_count))
            torch.cuda.empty_cache()
        
        print(f"mean accuracy across epoch: {statistics.mean(log)}")
        
        #print(f"Train Accuracy:{correct_num / train_count}")
        

  2%|▏         | 1/60 [00:48<47:41, 48.50s/it]

mean accuracy across epoch: 0.23629952203855162


  3%|▎         | 2/60 [01:34<45:17, 46.85s/it]

mean accuracy across epoch: 0.35294117647058826


  5%|▌         | 3/60 [02:20<44:15, 46.59s/it]

...

mean accuracy across epoch: 0.35294117647058826


 15%|█▌        | 9/60 [07:01<40:00, 47.07s/it]

mean accuracy across epoch: 0.35294117647058826


 17%|█▋        | 10/60 [07:48<39:16, 47.14s/it]

mean accuracy across epoch: 0.35294117647058826


In [ ]:
"""
__ = (torch.round(y_pred).to(torch.int64) == y)
preds = np.all(__.cpu().numpy(), axis=1)
correct_num += np.count_nonzero(preds)
"""

'\n__ = (torch.round(y_pred).to(torch.int64) == y)\npreds = np.all(__.cpu().numpy(), axis=1)\ncorrect_num += np.count_nonzero(preds)\n'